In [20]:
import random
import time

from rich.live import Live
from rich.table import Table


def generate_table() -> Table:
    """Make a new table."""
    table = Table()
    table.add_column("ID")
    table.add_column("Value")
    table.add_column("Status")

    for row in range(random.randint(2, 6)):
        value = random.random() * 100
        table.add_row(
            f"{row}", f"{value:3.2f}", "[red]ERROR" if value < 50 else "[green]SUCCESS"
        )
    return table


with Live(generate_table(), refresh_per_second=4) as live:
    for _ in range(40):
        time.sleep(0.4)
        live.update(generate_table())

Output()

KeyboardInterrupt: 

In [1]:
from pettingzoo.test import api_test
import Environment
env = Environment.VehicleJobSchedulingEnvACE()
# env = Environment.VehicleJobSchedulingEnvACE()
env.reset()
api_test(env, num_cycles=100)
para = env.parameters

Starting API test
Passed API test


/home/yuan/ResMan/man/lib/python3.9/site-packages/pettingzoo/test/api_test.py:94: UserWarning: Observation is not NumPy array
  warnings.warn("Observation is not NumPy array")
/home/yuan/ResMan/man/lib/python3.9/site-packages/pettingzoo/test/api_test.py:177: UserWarning: We recommend agents to be named in the format <descriptor>_<number>, like "player_0"
  warnings.warn(


In [2]:
para.cluster.machines[0].info()

{'reward': 0,
 'earning': 0,
 'slot': {'Resource 0': '██████████', 'Resource 1': '██████████'},
 'action': 0.6888888888888889,
 'bid': 25.33333333333333,
 'finished_job_num': 1,
 'running_job': []}